In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Hypterparameter

In [5]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop

## Loss function

In [6]:
loss_fn = nn.CrossEntropyLoss()

## Optimizer

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

* optimizer.zero_grad(): 매개변수 변화도 재설정
* loss.backwards(): prediction loss back prop.
* optimizer.step(): optimizes parameter using prediction loss

## 전체 구현

In [12]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # pred loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # loss backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 ==0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [13]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
    print("Done!")

Epoch 1
-------------------------------
loss: 2.136067  [   64/60000]
loss: 0.547209  [ 6464/60000]
loss: 0.397511  [12864/60000]
loss: 0.505387  [19264/60000]
loss: 0.425313  [25664/60000]
loss: 0.422986  [32064/60000]
loss: 0.374794  [38464/60000]
loss: 0.537816  [44864/60000]
loss: 0.480961  [51264/60000]
loss: 0.533268  [57664/60000]
Test Error: 
 Accuracy: 84.4%, Avg loss: 0.425661 

Done!
Epoch 2
-------------------------------
loss: 0.260509  [   64/60000]
loss: 0.366688  [ 6464/60000]
loss: 0.294957  [12864/60000]
loss: 0.395225  [19264/60000]
loss: 0.423893  [25664/60000]
loss: 0.368929  [32064/60000]
loss: 0.311007  [38464/60000]
loss: 0.495923  [44864/60000]
loss: 0.394137  [51264/60000]
loss: 0.456255  [57664/60000]
Test Error: 
 Accuracy: 85.2%, Avg loss: 0.395863 

Done!
Epoch 3
-------------------------------
loss: 0.216184  [   64/60000]
loss: 0.337523  [ 6464/60000]
loss: 0.239142  [12864/60000]
loss: 0.327971  [19264/60000]
loss: 0.369970  [25664/60000]
loss: 0.340270